#### passing parameters to the notebook (adding a widget)

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

#### invoke a notebook from another notebook using the %run command

In [0]:
%run "../Includes/configuration"

In [0]:
%run  "../Includes/common_functions"

In [0]:
races_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                   StructField("year", IntegerType(), True),
                                   StructField("round", IntegerType(), True),
                                   StructField("circuitId", IntegerType(), True),
                                   StructField("name", StringType(), True),
                                   StructField("date", DateType(), True),
                                   StructField("time", StringType(), True),
                                   StructField("url", StringType(), True)
                                   ])

In [0]:
race_df = spark.read\
    .option("header", True)\
    .schema(races_schema)\
    .csv(f"{raw_folder_path}/races.csv")

In [0]:
race_df= race_df.select(col("raceId").alias("race_id"),
                        col("year").alias("race_year"),
                        col("round"), col("circuitId").alias("circuit_id"),
                        col("name"), col("date"), col("time"))\
                        .withColumn("race_timestamp",concat(col("date"), lit(" "), col("time")))\
                        .withColumn("ingestion_date", current_timestamp())\
                        .withColumn("data_source", lit(v_data_source))\
                        .drop("date", "time")

In [0]:
race_df=race_df.drop("ingestion_date")

In [0]:
race_df.write.mode("overwrite").parquet(f"{processed_folder_path}/races")

In [0]:
dbutils.notebook.exit("Success")